In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
all = pd.read_csv('/content/drive/MyDrive/all_features.csv')

In [ ]:
selected = pd.read_csv('/content/drive/MyDrive/selected_features.csv')

In [ ]:
all.columns

Index(['DATE', 'GDP(십억원)', 'GNI(십억원)', 'GDP_GROWTH(%)', 'KOSPI', '국고채(3년)(%)',
       '국고채(10년)(%)', '뉴스심리지수', '경제심리지수', '수출금액지수', '수입금액지수', '수출물가지수(원화기준)',
       '수출물가지수(계약통화기준)', '수출물가지수(달러기준)', '수입물가지수(원화기준)', '수입물가지수(계약통화기준)',
       '수입물가지수(달러기준)', '대외채권', '대외채무', '순대외채권', '외환보유액(천달러)', '기준금리',
       'CD(91일)', '무담보콜금리(1일, 전체거래)', '경상수지(계절조정)', 'CPI', 'PPI',
       '원/미국달러(매매기준율)', '일본엔/달러', '달러/유로', '미국기준금리', 'NASDAQ', '미국CPI',
       '미국PPI', '본원 통화(달러 발행량)', '미국 소비자심리지수', '금', '은', '원유', '한국실업률',
       '미국실업률', '중국위안/달러', '장단기금리차'],
      dtype='object')

In [ ]:
selected.columns

Index(['DATE', 'KOSPI', '기준금리', '수출물가지수(원화기준)', '수입물가지수(원화기준)', 'PPI', '한국실업률',
       '금', '은', '원유', '중국위안/달러', 'NASDAQ', '외환보유액(천달러)', '미국PPI', '뉴스심리지수',
       '대외채무', '본원 통화(달러 발행량)', 'CPI', '장단기금리차', 'GDP_GROWTH(%)',
       '원/미국달러(매매기준율)'],
      dtype='object')

In [ ]:
%%capture
!pip install catboost

In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
for col in selected.columns:
    if col != 'DATE' and selected[col].dtype == 'object':
        selected[col] = selected[col].str.replace(',', '')
        selected[col] = pd.to_numeric(selected[col], errors='coerce')

selected catboost

In [ ]:
# DATE가 object 타입일 수 있으니, 먼저 datetime으로 변환
selected['DATE'] = pd.to_datetime(selected['DATE'], errors='coerce')

# 결측치가 아닌 값들만 datetime 파싱됐는지 확인
print("DATE 결측치 수:", selected['DATE'].isna().sum())

# Year, Month, Quarter 컬럼 생성
selected['Year'] = selected['DATE'].dt.year
selected['Month'] = selected['DATE'].dt.month
selected['Quarter'] = selected['DATE'].dt.quarter

# DATE 컬럼 제거
selected = selected.drop(columns=['DATE'])

# 결과 확인
print(selected[['Year', 'Month', 'Quarter']].isna().sum())

DATE 결측치 수: 0
Year       0
Month      0
Quarter    0
dtype: int64


In [ ]:
# 타겟과 피처 지정
target = '원/미국달러(매매기준율)'
X = selected.drop(columns=[target])
y = selected[target]

In [ ]:
# 학습/테스트 분리 (시계열 고려, 셔플 X)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

In [ ]:
# 모델 학습
model = CatBoostRegressor(verbose=0)
model.fit(X_train, y_train)

In [ ]:
# 예측 및 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Selected Catboost")
print(f"MSE: {mse:}")
print(f"RMSE: {rmse:}")
print(f"R² Score: {r2:}")

Selected Catboost
MSE: 35735.99060485963
RMSE: 189.03965352502007
R² Score: -3.128130419256461


all catboost

In [ ]:
for col in all.columns:
    if col != 'DATE' and all[col].dtype == 'object':
        all[col] = all[col].str.replace(',', '')
        all[col] = pd.to_numeric(all[col], errors='coerce')

In [ ]:
# DATE가 object 타입일 수 있으니, 먼저 datetime으로 변환
all['DATE'] = pd.to_datetime(all['DATE'], errors='coerce')

# 결측치가 아닌 값들만 datetime 파싱됐는지 확인
print("DATE 결측치 수:", all['DATE'].isna().sum())

# Year, Month, Quarter 컬럼 생성
all['Year'] = all['DATE'].dt.year
all['Month'] = all['DATE'].dt.month
all['Quarter'] = all['DATE'].dt.quarter

# DATE 컬럼 제거
all = all.drop(columns=['DATE'])

# 결과 확인
print(all[['Year', 'Month', 'Quarter']].isna().sum())


DATE 결측치 수: 0
Year       0
Month      0
Quarter    0
dtype: int64


In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

# 타겟 설정
target_col = '원/미국달러(매매기준율)'

# all DataFrame 사용
X = all.drop(columns=[target_col])  # DATE는 예측에 사용하지 않음
y = all[target_col]

# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# CatBoostRegressor 모델 생성 및 학습
model = CatBoostRegressor(verbose=0, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# 예측 및 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("All Catboost")
print(f"MSE: {mse:}")
print(f"RMSE: {rmse:}")
print(f"R² Score: {r2:}")

All Catboost
MSE: 34749.56438015213
RMSE: 186.41235039597598
R² Score: -3.0141809796118917
